# Feature Engineering & Ensemble Model

## 목표
- Feature Engineering으로 성능 향상
- 앙상블 기법으로 Macro-F1 > 0.9 달성
- 최적 하이퍼파라미터 탐색

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# Hyperparameter tuning
import optuna
from optuna.samplers import TPESampler

from tqdm import tqdm
import time
import gc

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
optuna.logging.set_verbosity(optuna.logging.WARNING)

print('라이브러리 로드 완료')

## 1. 데이터 로드

In [ ]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

# 기본 피처
feature_cols = [col for col in train.columns if col not in ['ID', 'target']]
X_train_orig = train[feature_cols]
y_train = train['target']
X_test_orig = test[feature_cols]

print(f"Original features: {len(feature_cols)}")
print(f"Train shape: {X_train_orig.shape}")
print(f"Test shape: {X_test_orig.shape}")

## 2. Advanced Feature Engineering

In [ ]:
def create_features(df):
    """고급 피처 엔지니어링"""
    df_new = df.copy()
    
    # 1. 통계적 특징
    df_new['row_mean'] = df.mean(axis=1)
    df_new['row_std'] = df.std(axis=1)
    df_new['row_max'] = df.max(axis=1)
    df_new['row_min'] = df.min(axis=1)
    df_new['row_median'] = df.median(axis=1)
    df_new['row_range'] = df_new['row_max'] - df_new['row_min']
    df_new['row_skew'] = df.skew(axis=1)
    df_new['row_kurt'] = df.kurtosis(axis=1)
    
    # 2. 비율 특징
    df_new['mean_to_std_ratio'] = df_new['row_mean'] / (df_new['row_std'] + 1e-10)
    df_new['max_to_min_ratio'] = df_new['row_max'] / (df_new['row_min'] + 1e-10)
    
    # 3. 상위 피처 간 상호작용 (중요도가 높은 피처들)
    important_features = ['X_01', 'X_02', 'X_03', 'X_04', 'X_05']  # 예시
    for i, feat1 in enumerate(important_features):
        if feat1 in df.columns:
            for feat2 in important_features[i+1:]:
                if feat2 in df.columns:
                    # 곱셈
                    df_new[f'{feat1}_mul_{feat2}'] = df[feat1] * df[feat2]
                    # 차이
                    df_new[f'{feat1}_diff_{feat2}'] = df[feat1] - df[feat2]
                    # 비율
                    df_new[f'{feat1}_ratio_{feat2}'] = df[feat1] / (df[feat2] + 1e-10)
    
    # 4. 분위수 특징
    df_new['row_q25'] = df.quantile(0.25, axis=1)
    df_new['row_q75'] = df.quantile(0.75, axis=1)
    df_new['row_iqr'] = df_new['row_q75'] - df_new['row_q25']
    
    # 5. 카운트 특징
    df_new['positive_count'] = (df > 0).sum(axis=1)
    df_new['negative_count'] = (df < 0).sum(axis=1)
    df_new['zero_count'] = (df == 0).sum(axis=1)
    
    # 6. 극값 특징
    df_new['abs_max'] = df.abs().max(axis=1)
    df_new['abs_min'] = df.abs().min(axis=1)
    df_new['abs_mean'] = df.abs().mean(axis=1)
    
    print(f"Features created: {len(df_new.columns) - len(df.columns)}")
    print(f"Total features: {len(df_new.columns)}")
    
    return df_new

# 피처 생성
X_train_fe = create_features(X_train_orig)
X_test_fe = create_features(X_test_orig)

In [ ]:
# PCA 특징 추가
n_components = 20
pca = PCA(n_components=n_components, random_state=RANDOM_STATE)

# 원본 피처에 대해 PCA 적용
X_train_pca = pca.fit_transform(X_train_orig)
X_test_pca = pca.transform(X_test_orig)

# PCA 컴포넌트를 데이터프레임에 추가
for i in range(n_components):
    X_train_fe[f'pca_{i}'] = X_train_pca[:, i]
    X_test_fe[f'pca_{i}'] = X_test_pca[:, i]

print(f"\nPCA variance explained: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Final feature count: {X_train_fe.shape[1]}")

## 3. Feature Selection

In [ ]:
# 피처 중요도 기반 선택
# LightGBM으로 피처 중요도 계산
lgb_selector = LGBMClassifier(
    n_estimators=100,
    random_state=RANDOM_STATE,
    verbose=-1,
    n_jobs=-1
)

lgb_selector.fit(X_train_fe, y_train)

# 피처 중요도
feature_importance = pd.DataFrame({
    'feature': X_train_fe.columns,
    'importance': lgb_selector.feature_importances_
}).sort_values('importance', ascending=False)

# 상위 피처 선택 (중요도 > 0인 피처들)
selected_features = feature_importance[feature_importance['importance'] > 0]['feature'].tolist()

print(f"Selected features: {len(selected_features)} / {len(X_train_fe.columns)}")
print(f"\nTop 20 features:")
display(feature_importance.head(20))

# 선택된 피처만 사용
X_train_selected = X_train_fe[selected_features]
X_test_selected = X_test_fe[selected_features]

## 4. 데이터 준비

In [ ]:
# Train/Validation 분할
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_selected, y_train,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_train
)

# 스케일링
scaler = RobustScaler()  # 이상치에 강한 스케일러
X_tr_scaled = scaler.fit_transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

print(f"Train shape: {X_tr_scaled.shape}")
print(f"Validation shape: {X_val_scaled.shape}")

## 5. Optuna 하이퍼파라미터 튜닝

In [ ]:
def objective_lgb(trial):
    """LightGBM 하이퍼파라미터 최적화"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbose': -1,
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'class_weight': 'balanced'
    }
    
    model = LGBMClassifier(**params)
    
    # 3-fold CV
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(
        model, X_tr_scaled, y_tr,
        cv=skf,
        scoring=make_scorer(f1_score, average='macro'),
        n_jobs=-1
    )
    
    return scores.mean()

# Optuna 스터디 생성 및 최적화
print("LightGBM 하이퍼파라미터 최적화 중...")
study_lgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_STATE)
)

study_lgb.optimize(
    objective_lgb,
    n_trials=30,  # 시간 절약을 위해 30번만
    show_progress_bar=True
)

print(f"\nBest LightGBM score: {study_lgb.best_value:.4f}")
print("Best parameters:")
print(study_lgb.best_params)

In [ ]:
def objective_xgb(trial):
    """XGBoost 하이퍼파라미터 최적화"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 10.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }
    
    model = XGBClassifier(**params)
    
    # 3-fold CV
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(
        model, X_tr_scaled, y_tr,
        cv=skf,
        scoring=make_scorer(f1_score, average='macro'),
        n_jobs=-1
    )
    
    return scores.mean()

# XGBoost 최적화
print("\nXGBoost 하이퍼파라미터 최적화 중...")
study_xgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_STATE)
)

study_xgb.optimize(
    objective_xgb,
    n_trials=30,
    show_progress_bar=True
)

print(f"\nBest XGBoost score: {study_xgb.best_value:.4f}")
print("Best parameters:")
print(study_xgb.best_params)

## 6. 최적화된 모델 학습

In [ ]:
# 최적 파라미터로 모델 생성
best_lgb = LGBMClassifier(
    **study_lgb.best_params,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1,
    objective='multiclass',
    metric='multi_logloss',
    class_weight='balanced'
)

best_xgb = XGBClassifier(
    **study_xgb.best_params,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    objective='multi:softprob',
    eval_metric='mlogloss',
    use_label_encoder=False
)

# CatBoost (자동 하이퍼파라미터 튜닝)
best_cat = CatBoostClassifier(
    iterations=800,
    depth=10,
    learning_rate=0.05,
    random_state=RANDOM_STATE,
    verbose=False,
    auto_class_weights='Balanced',
    task_type='CPU',
    l2_leaf_reg=5
)

# 개별 모델 학습 및 평가
models = {
    'LightGBM': best_lgb,
    'XGBoost': best_xgb,
    'CatBoost': best_cat
}

model_scores = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_tr_scaled, y_tr)
    
    # Validation 평가
    y_pred = model.predict(X_val_scaled)
    score = f1_score(y_val, y_pred, average='macro')
    model_scores[name] = score
    
    print(f"{name} Validation Macro F1: {score:.4f}")

print("\n" + "="*50)
print("Model Performance Summary:")
for name, score in model_scores.items():
    print(f"{name}: {score:.4f}")

## 7. 앙상블 모델

In [ ]:
# Soft Voting Ensemble
voting_clf = VotingClassifier(
    estimators=[
        ('lgb', best_lgb),
        ('xgb', best_xgb),
        ('cat', best_cat)
    ],
    voting='soft',
    n_jobs=-1
)

print("Training Voting Ensemble...")
voting_clf.fit(X_tr_scaled, y_tr)

# 평가
y_pred_voting = voting_clf.predict(X_val_scaled)
voting_score = f1_score(y_val, y_pred_voting, average='macro')

print(f"\nVoting Ensemble Validation Macro F1: {voting_score:.4f}")

In [ ]:
# Stacking Ensemble
base_models = [
    ('lgb', best_lgb),
    ('xgb', best_xgb),
    ('cat', best_cat)
]

# Meta learner
meta_learner = LogisticRegression(
    max_iter=1000,
    random_state=RANDOM_STATE,
    multi_class='multinomial',
    solver='lbfgs'
)

stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=3,  # Cross-validation for generating meta features
    n_jobs=-1,
    passthrough=False  # Don't include original features
)

print("\nTraining Stacking Ensemble...")
stacking_clf.fit(X_tr_scaled, y_tr)

# 평가
y_pred_stacking = stacking_clf.predict(X_val_scaled)
stacking_score = f1_score(y_val, y_pred_stacking, average='macro')

print(f"\nStacking Ensemble Validation Macro F1: {stacking_score:.4f}")

## 8. 최종 모델 선택 및 예측

In [ ]:
# 모든 모델 점수 비교
all_scores = model_scores.copy()
all_scores['Voting Ensemble'] = voting_score
all_scores['Stacking Ensemble'] = stacking_score

# 최고 모델 선택
best_model_name = max(all_scores, key=all_scores.get)
best_score = all_scores[best_model_name]

print("\n" + "="*60)
print("Final Model Selection")
print("="*60)
for name, score in sorted(all_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{name}: {score:.4f}")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Validation Macro F1: {best_score:.4f}")

if best_score >= 0.9:
    print("\n✅ 목표 달성! (Macro F1 >= 0.9)")
else:
    print(f"\n⚠️ 목표에 근접 (현재: {best_score:.4f}, 목표: 0.9)")

In [ ]:
# 최종 모델로 전체 학습 데이터로 재학습
if best_model_name == 'Voting Ensemble':
    final_model = voting_clf.__class__(estimators=voting_clf.estimators, voting='soft', n_jobs=-1)
elif best_model_name == 'Stacking Ensemble':
    final_model = stacking_clf
else:
    final_model = models[best_model_name]

print(f"\nTraining final {best_model_name} on full training data...")
final_model.fit(X_train_scaled, y_train)

# 테스트 데이터 예측
test_predictions = final_model.predict(X_test_scaled)

print("\n예측 완료!")
print("\n예측 분포:")
pred_distribution = pd.Series(test_predictions).value_counts().sort_index()
for cls, count in pred_distribution.items():
    print(f"Class {cls}: {count:6d} ({count/len(test_predictions)*100:5.2f}%)")

In [ ]:
# 제출 파일 생성
submission['target'] = test_predictions

# 타임스탬프 추가
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

submission_filename = f'../submissions/advanced_{best_model_name.lower().replace(" ", "_")}_f1_{best_score:.4f}_{timestamp}.csv'
submission.to_csv(submission_filename, index=False)

print(f"\n✅ 제출 파일 생성 완료: {submission_filename}")
print("\n제출 파일 미리보기:")
display(submission.head(10))

## 9. 추가 개선 방안

In [ ]:
print("="*60)
print("📊 성능 분석 및 추가 개선 방안")
print("="*60)

if best_score >= 0.9:
    print("""
✅ 목표 성능 달성!

추가 개선 아이디어:
1. **Pseudo Labeling**: Test 데이터의 고신뢰도 예측을 학습에 활용
2. **더 많은 앙상블**: Neural Network 등 다른 모델 추가
3. **Post-processing**: Threshold 최적화
4. **Test Time Augmentation**: 여러 변형 데이터로 예측 후 평균
5. **더 깊은 Feature Engineering**: 
   - Frequency domain features (FFT)
   - Clustering features
   - Target encoding
    """)
else:
    gap = 0.9 - best_score
    print(f"""
현재 성능: {best_score:.4f}
목표까지 차이: {gap:.4f}

개선 전략:
1. **더 많은 Optuna trials**: 현재 30 → 100+
2. **Neural Network 추가**: TabNet, Deep Neural Network
3. **클래스별 최적화**: 성능이 낮은 클래스 집중 개선
4. **데이터 증강**: SMOTE, ADASYN 등
5. **더 복잡한 앙상블**: Multi-level stacking
6. **Feature Selection 재검토**: 다른 방법 시도
    """)

print(f"\n현재 최종 성능: {best_score:.4f} ({best_model_name})")
print("\n다음 노트북에서 추가 실험을 진행하세요.")